In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [7]:
import numpy as np
import string
from nltk.corpus import stopwords
import math

def softmax(z):
    e_z = np.exp(z)
    sum_e_z = np.sum(e_z)
    return e_z / sum_e_z

class word2vec(object):
    def __init__(self):
        self.N = 300
        self.X_train = []
        self.y_train = []
        self.window_size = 2
        self.alpha = 0.003
        self.words = []
        self.word_index = {}
        self.loss = 0

    def initialize(self,V,data):
        self.V = V
        self.W1 = np.random.uniform(-1, 1, (self.N, self.V))
        self.W2 = np.random.uniform(-1, 1, (self.V, self.N))
        self.b1 = np.random.uniform(-1, 1, (self.N, 1))
        self.b2 = np.random.uniform(-1, 1, (self.V, 1))

          
        self.words = data
        for i in range(len(data)):
            self.word_index[data[i]] = i
  
      
    def feed_forward(self,X):
#         print(X.shape)
        X = X.reshape(self.V,1)
        self.h = np.dot(self.W1,X) + self.b1
        self.u = np.dot(self.W2 ,self.h) + self.b2
        #print(self.u)
        self.y_hat = softmax(self.u)
        return self.y_hat
          
    def backpropagate(self,x,t):
        t = t.reshape(self.V,1)
        x = x.reshape(self.V,1)
        self.grad_b2 = self.y_hat - t
        self.grad_W2 = np.dot(self.y_hat - t, self.h.T)
        self.grad_b1 = np.dot(self.W2.T, self.y_hat - t)
        self.grad_W1 = np.dot(np.dot(self.W2.T, self.y_hat - t), x.T)
        self.W1 = self.W1 - self.alpha * self.grad_W1
        self.W2 = self.W2 - self.alpha * self.grad_W2
        self.b1 = self.b1 - self.alpha * self.grad_b1
        self.b2 = self.b2 - self.alpha * self.grad_b2
#         return(self.W1_new , self.W2_new , self.b1_new, self.b2_new)


    def train(self,epochs):
        for x in range(1,epochs):       
            self.loss = 0
            for j in range(len(self.X_train)):
                if(len(self.X_train[j])==0):
                    continue
#                 print(j)
                self.y_hat = self.feed_forward(self.X_train[j])
                self.backpropagate(self.X_train[j],self.y_train[j])
                C = 0
                for m in range(self.V):
                    if(self.y_train[j][m]):
                        self.loss += -1*self.u[m][0]
                        C += 1
                self.loss += C*np.log(np.sum(np.exp(self.u)))
                if(j%1000==0):
                    print("epoch ",x, " loss = ",self.loss)
            self.alpha *= 1/( (1+self.alpha*x) )
             
#     def predict(self,word,number_of_predictions):
#         if word in self.words:
#             index = self.word_index[word]
#             X = [0 for i in range(self.V)]
#             X[index] = 1
#             prediction = self.feed_forward(X)
#             output = {}
#             for i in range(self.V):
#                 output[prediction[i][0]] = i

#             top_context_words = []
#             for k in sorted(output,reverse=True):
#                 top_context_words.append(self.words[output[k]])
#                 if(len(top_context_words)>=number_of_predictions):
#                     break
      
#             return top_context_words
#         else:
#             print("Word not found in dictionary")

In [2]:
def preprocessing(sentence):
    stop_words = set(stopwords.words('english'))   
    training_data = []
#     sentences = corpus.split(".")
#     for i in range(len(sentences)):
#         sentences[i] = sentences[i].strip()
#         sentence = sentences[i].split()
    x = [word.strip(string.punctuation) for word in sentence
                                     if word not in stop_words]
    x = [word for word in x if len(word)!=0]
    x = [word.lower() for word in x]
#     training_data.append(x)
    return x

def index_word_maps(data: list) -> tuple:
    
    words = sorted(list(set(data)))
    
    word_to_index = {word: index for index, word in enumerate(words)}
    index_to_word = {index: word for index, word in enumerate(words)}
    return word_to_index, index_to_word


def word_to_one_hot_vector(word: str, word_to_index: dict, vocabulary_size: int) -> np.ndarray:
    
    one_hot_vector = np.zeros(vocabulary_size)
    one_hot_vector[word_to_index[word]] = 1
    return one_hot_vector

def context_words_to_vector(context_words: list,
                            word_to_index: dict) -> np.ndarray:
    vocabulary_size = len(word_to_index)
    context_words_vectors = [
        word_to_one_hot_vector(word, word_to_index, vocabulary_size)
        for word in context_words]
    return np.mean(context_words_vectors, axis=0)

In [ ]:

# print(corpus.split(".")[1])
preprocessing(corpus)

training_data = preprocessing(corpus)
epochs = 3

# X_train, Y_train = prepare_data_for_training(training_data,w2v)

In [ ]:
training_data

In [3]:
from nltk.corpus import gutenberg
import nltk
fields = nltk.corpus.gutenberg.fileids()

words = []
for field in fields:
    words += list(gutenberg.words(field))

In [ ]:
gutenberg.sents("shakespeare-macbeth.txt")[0]

In [4]:
stop_words = set(stopwords.words('english') + ["[","]","''"])
words = [word.strip(string.punctuation) for word in words
                                     if word not in stop_words and len(word)!=0]
words = [word.lower() for word in words]

In [ ]:
word_to_index, index_to_word = index_word_maps(words)
word_to_index

In [8]:
w2v = word2vec()
w2v.initialize(len(word_to_index.keys()),words)

In [9]:
training_data = gutenberg.sents('shakespeare-macbeth.txt')

for m in range(0,len(training_data)):
    sent = training_data[m]
    sent = preprocessing(sent)
#     w2v.X_train = []
#     w2v.y_train = []

    for i in range(0,len(sent)):
        context_words = []
        train_word = word_to_one_hot_vector(sent[i],word_to_index,len(word_to_index.keys()))
        for j in range(i-w2v.window_size,i+w2v.window_size):
                if i!=j and j>=0 and j<len(sent):
                    context_words.append(sent[j])
        
        if len(context_words) > 0:
            w2v.y_train.append(train_word)
            w2v.X_train.append(context_words_to_vector(context_words,word_to_index))

In [ ]:
preprocessing(training_data[5])

In [10]:
w2v.train(10)

epoch  1  loss =  21.855757991734023
epoch  1  loss =  23752.33259597954
epoch  1  loss =  42723.44366991852
epoch  1  loss =  60068.39626752783
epoch  1  loss =  78027.99396048703
epoch  1  loss =  95147.45370790405
epoch  1  loss =  112496.25620092622
epoch  1  loss =  129368.04053148233


KeyboardInterrupt: 

In [ ]:
len(w2v.y_train)

In [ ]:
w2v.X_train[15]